In [1]:
using CSV, DataFramesMeta, PlotlyJS, FloatingTableView, Dates

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17619419689152091585\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-3896012666797303654\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17619419689152091585\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
df = CSV.File("spec20210723.tsv") |> DataFrame;

In [ ]:
# FloatingTableView package의 browse 함수를 이용하면 dataframe을 별도의 창에 띄울 수 있다.
# browse(df)

In [3]:
# PRODUCT_CODE를 string type으로 바꾸고 PLANT 컬럼과 합쳐서 Plantcode 컬럼에 저장하기
@transform! df begin    
    :Plantcode = :PLANT .* string.(:PRODUCT_CODE)
end;

In [4]:
# EFFECTIVITY_DATE 컬럼을 나누어서 date에 해당하는 값만 취한 후 날짜 타입 데이터로 변경하기
# missing value 처리를 할 수 없어서 broadcasting을 못하고 for loop를 사용함
# df[!, :e_date] = 
#   [ismissing(df[idx, :EFFECTIVITY_DATE]) ? 
#     missing : 
#     Date(split(df[idx, :EFFECTIVITY_DATE], " ")[1],"mm/dd/yyyy") 
#     for idx in 1:size(df)[1]];

In [9]:
# 위 코드에 대한 vectorization expression.  passmissing 함수를 찾아내어 성공함
# 문자열 EFFECTIVITY_DATE 컬럼을 split 함수를 이용하여 공백을 기준으로 나누고
# 나누어진 첫번째 컬럼(날짜에 대한 문자열)을 date 데이터 타입으로 변경한다.
@transform! df @byrow @passmissing :e_date = Date(split(:EFFECTIVITY_DATE, ' ')[1], "mm/dd/yyyy");

In [6]:
sort!(df, [:Plantcode, :e_date]);

In [8]:
# grouped dataframe의 맨 마지막 row를 뽑아내는 표현
gdf = groupby(df, :Plantcode)
dfl = combine(last, gdf);

# 다원화 관계의 제품 묶기

다음 컬럼들의 값이 동등하다면 다원화 관계로 묶을 수 있다.
- SIZE_FULL, PATTERN, PLY_RATING, BW_INDICATOR, BRAND
- PLY_LOCK, PRE_BELT_TYPE
- MOLD_OD, MOLD_SH, MOLD_SW, MOLD_SD, MOLD_RW, MOLD_TDW
- MOLD_TR1, MOLD_TR2, RPB_TYPE
- CTB_COMPOUND, SUT_COMPOUND

Studding 여부, foam의 부착 여부에 대한 구분을 TL_INDICATOR에 표시하기 시작하였다.  
다원화 관계를 정의할 때, 이를 어떻게 처리할 지 결정이 필요하다.

다음 컬럼들은 위의 기준에 포함시킬 지 여부를 고민해야 한다.
- OVER
- C01_ROLLED, C02_ROLLED, BT1_ROLLED, BT2_ROLLED
- C01_MATERIAL, C02_MATERIAL, C03_MATERIAL, BT1_MATERIAL
- PRE_BT1_PART_CODE3, PRE_JLB_JLC_PART_CODE3
- MOLD_CODE

다음 컬럼의 값이 다르면 무조건 다원화 관계가 아니다.
- ORIGINAL_CUSTOMER